In [ ]:

import numpy as np
from infromation_theory_utils import JSD, Entropy
import pandas as pd
import soundfile as sf
import os
from scipy.linalg import svd, toeplitz
from scipy.stats import zscore
from statsmodels.tsa.stattools import acf
from scipy.signal import spectrogram
import time
    
import matplotlib.pyplot as plt
%matplotlib inline

# Import packages related to downsampling 
import samplerate 
import resampy
import scipy.signal as scipy_signal
import math

# Import packages related to filtering
from scipy.io import wavfile
import os
import matplotlib.pyplot as plt
import aasi_filters as filters
from scipy.signal import butter, lfilter

In [115]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import butter, lfilter
import scipy.signal as scipy_signal
import aasi_filters as filters
from scipy import signal
import samplerate

def sine_signal(freq): 
    T = 10/freq # To visualize 10 cycles of the freq Hz sine wave in 1 second
    Fs = 384000
    Ts = 1./Fs
    N = int(T/Ts)

    t = np.linspace(0, T, N)
    sig = np.sin(2*np.pi*freq*t)
#     plt.plot(t, sig)
    return sig
#     return y 
#     plt.show()

def low_pass(sig):
    # Low pass filter
    is_stereo = len(sig.shape) == 2
    sosfilter = filters.design_lowpass_butter_filter(20000, 16, 384000, sos=True)
    sig = filters.filter(sig, sosfilter, stereo=is_stereo)
    return sig

# freqs = [15000, 20000, 22000]

def power_signal(sig): 
    freqs, psd = signal.welch(sig, nperseg=None)
    
#     Plotting # 
#     plt.figure(figsize=(5, 4))
#     plt.semilogx(freqs, psd)
#     plt.title('PSD: power spectral density')
#     plt.xlabel('Frequency')
#     plt.ylabel('Power')
#     plt.tight_layout()
    
    # Calculate # 
#     print(psd)
    print(sum(psd)/len(psd)) 
    print(max(psd))
    
# DOWNSAMPLING #
# Need to redo: 
def downsampling(sig, original_freq, final_freq): 
    rate_ratio = original_freq/final_freq
    conversion_method = 'sinc_best'
    
    # for resample
#     print(len(sig)*(1/rate_ratio))
#     print(int(len(sig)*(1/rate_ratio)))
    
    # Need to figure out the frequency 
    sig = scipy_signal.resample(sig, int(len(sig)*(1/rate_ratio)))
#     print(len(sig))
    
    # signal is sampled int(len(sig)*rate_ratio times) 
    return sig

def power_graphs(freq): 
    print('Original: ')
    power_signal(sine_signal(freq)) 
    sig_1 = low_pass(sine_signal(freq))
    print('After Low-pass filter: ')
    power_signal(sig_1) 
    sig_2 = downsampling(sig_1, 384000, 44100)
    print('After Downsampling: ')
    power_signal(sig_2) 
    
# Plot the low pass filter

# b, a = signal.butter(4, 100, 'low', analog=True)
# >>> w, h = signal.freqs(b, a)
# >>> plt.semilogx(w, 20 * np.log10(abs(h)))
# >>> plt.title('Butterworth filter frequency response')
# >>> plt.xlabel('Frequency [radians / second]')
# >>> plt.ylabel('Amplitude [dB]')
# >>> plt.margins(0, 0.1)
# >>> plt.grid(which='both', axis='both')
# >>> plt.axvline(100, color='green') # cutoff frequency
# >>> plt.show()

print('Freq = 15kHz')
power_graphs(15000)
print()

print('Freq = 20kHz')
power_graphs(20000)
print()

print('Freq = 22kHz')
power_graphs(22000)
print()

# print('Freq = 21750Hz')
# power_graphs(21750) # (384000/(256/29))/2 or f =15kHz
# print()

# print('Freq = 22000')
# power_graphs(22000) # (384000/8.72727272727)/2 or f =20kHz
# print()

# print('Freq = 21333.3333333Hz')
# power_graphs(21333) # (384000/(153/17))/2 or f =25kHz
# print()

print('Freq = 25kHz')
power_graphs(25000)

# Optional: plot signal in time domain and frequency domain

Freq = 15kHz
Original: 
0.9922479692884071
85.16501864713052
After Low-pass filter: 
0.9918228682351709
85.22141977451781
After Downsampling: 
0.9664739055006386
9.653988958832098

Freq = 20kHz
Original: 
0.9896906028925807
63.77485664307893
After Low-pass filter: 
0.202095442622087
12.929883195298
After Downsampling: 
0.15992151861614357
1.4992787449333569

Freq = 22kHz
Original: 
0.9886362351353956
57.75129048313462
After Low-pass filter: 
0.0011287808534625953
0.06284842025130875
After Downsampling: 
0.00011711757811812538
0.001057105937331451

Freq = 25kHz
Original: 
0.9935063510338327
50.7167417593429
After Low-pass filter: 
1.6584157580306126e-05
0.0006404090786699185
After Downsampling: 
1.1832252399717837e-05
5.5513967732614274e-05


/anaconda3/lib/python3.6/site-packages/scipy/signal/spectral.py:1963: UserWarning: nperseg = 256 is greater than input length  = 29, using nperseg = 29
  .format(nperseg, input_length))
/anaconda3/lib/python3.6/site-packages/scipy/signal/spectral.py:1963: UserWarning: nperseg = 256 is greater than input length  = 192, using nperseg = 192
  .format(nperseg, input_length))
/anaconda3/lib/python3.6/site-packages/scipy/signal/spectral.py:1963: UserWarning: nperseg = 256 is greater than input length  = 22, using nperseg = 22
  .format(nperseg, input_length))
/anaconda3/lib/python3.6/site-packages/scipy/signal/spectral.py:1963: UserWarning: nperseg = 256 is greater than input length  = 174, using nperseg = 174
  .format(nperseg, input_length))
/anaconda3/lib/python3.6/site-packages/scipy/signal/spectral.py:1963: UserWarning: nperseg = 256 is greater than input length  = 19, using nperseg = 19
  .format(nperseg, input_length))
/anaconda3/lib/python3.6/site-packages/scipy/signal/spectral.py:19

In [40]:
lag = 512
cotas = pd.read_csv('./boundary_files/Cotas_HxC_bins_' + str(int(lag)) + '.csv')
noise = pd.read_csv('./coloredNoises/coloredNoises_' + str(int(lag)) + '.csv')

# Known files with rain in them
datapath = "./Rain/" 

downsampling_rate = 44100
original_rate = 384000
conversion_method = 'sinc_best'
rate_ratio = downsampling_rate/original_rate 
# print(rate_ratio)
x = zscore(x)

NameError: name 'pd' is not defined

In [ ]:
listing = os.listdir(datapath)

H = []
C = []
labels = []

#additional functionality to 
#f = open("./ECIComputerTimes.txt","w+")

for file in listing:
#    start = time.time()
    x, fs = sf.read(datapath + file)
    x = zscore(x) 
    
    ### FILTERING ###
    fcutoff = 22050
    order = 2 
    is_stereo = len(x.shape) == 2
    sosfilter = filters.design_lowpass_butter_filter(fcutoff, order, fs, sos=True)
    filt_data_sos = filters.filter(x, sosfilter, stereo=is_stereo)
    x = filt_data_sos

    ### DOWNSAMPLING ### 
    # 1) Samplerate 
    # https://pypi.org/project/samplerate/

    # Change the rate_ratio here: 
    downsampled_signal = samplerate.resample(x, rate_ratio, conversion_method, 1)
    x = downsampled_signal

    # Algorithm steps 
    #Auto-Correlation Matrix
    rxx = acf(x, nlags=lag, unbiased=True, fft=True)
    Sxx = toeplitz(rxx)

    #Singular Value Decomposition
    U, s, Vt = svd(Sxx)

    H.append(Entropy(s))              # Entropy
    C.append(Entropy(s)*JSD(s))       # Complexity, also called EGCI index
    labels.append(file.split('.')[0])  # Species name

#    end = time.time()
#    f.write(file + ": " + str(round(end - start,3)) + "\n")
#    f.close()